In [9]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    temperature=0,
    format="json",
    model="llama3.1:latest"
)

In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
response = llm.invoke(messages)
print(response.content)

The translation of "I love programming" in French is:

"J'adore le programmation."


In [15]:
from langchain.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    """Add two integers."""
    return a + b

In [16]:
llm_with_tools = llm.bind_tools([add_numbers])

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful and may use tools."),
    ("human", "{input}")
])

chain = prompt | llm_with_tools

In [20]:
response = chain.invoke({"input": "What is 12 + 30?"})
print("MODEL OUTPUT:")
print(response)

MODEL OUTPUT:
content='' additional_kwargs={} response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-11-13T18:08:10.708481Z', 'done': True, 'done_reason': 'stop', 'total_duration': 71713102700, 'load_duration': 63700720300, 'prompt_eval_count': 172, 'prompt_eval_duration': 1677572500, 'eval_count': 23, 'eval_duration': 6268309800, 'model_name': 'llama3.1:latest', 'model_provider': 'ollama'} id='lc_run--88f186b4-a42d-4f88-9ab7-758ac12560fc-0' tool_calls=[{'name': 'add_numbers', 'args': {'a': 12, 'b': 30}, 'id': 'b5d3baf4-f228-47f3-9aaa-17284f412046', 'type': 'tool_call'}] usage_metadata={'input_tokens': 172, 'output_tokens': 23, 'total_tokens': 195}


In [21]:
if response.tool_calls:
    call = response.tool_calls[0]
    result = add_numbers.invoke(call["args"])
    print("TOOL RESULT:", result)

TOOL RESULT: 42


In [12]:
print(response)

content='' additional_kwargs={} response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-11-13T17:59:28.3236635Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8986444700, 'load_duration': 617100700, 'prompt_eval_count': 173, 'prompt_eval_duration': 1888862000, 'eval_count': 23, 'eval_duration': 6289215300, 'model_name': 'llama3.1:latest', 'model_provider': 'ollama'} id='lc_run--6870f271-83f7-4c6c-be03-d5540f1bf4ea-0' tool_calls=[{'name': 'add_numbers', 'args': {'a': 12, 'b': 30}, 'id': '17344f7e-1d09-4092-990b-b7dd49adf931', 'type': 'tool_call'}] usage_metadata={'input_tokens': 173, 'output_tokens': 23, 'total_tokens': 196}


In [22]:
!pip list | findstr langchain

langchain                1.0.5
langchain-classic        1.0.0
langchain-community      0.4.1
langchain-core           1.0.4
langchain-ollama         1.0.0
langchain-openai         1.0.2
langchain-text-splitters 1.0.0


In [87]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage


llm = ChatOllama(
    temperature=0,
    format="json",
    model="llama3.1:latest"
)

# ... (Tools and LLM setup as before) ...
# 1. Define a tool using the @tool decorator
# The docstring becomes the tool's description, which Llama 3.1 uses for reasoning.

@tool
def get_weather(city: str):
    """Fetches weather for given city"""
    weather_data = { # Example data, use API in real-world
        "New York": "Sunny, 25°C", 
        "London": "Cloudy, 18°C", 
        "Tokyo": "Rainy, 22°C"
        }
    return weather_data.get(city, "Weather data not available.")

@tool
def multiply(x: float, y: float) -> float:
    """Multiplies two numbers and returns the result."""
    return x * y

@tool
def power_of_two(x: float) -> float:
    """Calculates the square of a number."""
    return x ** 2

# List of tools to bind to the model
tools = [get_weather]


prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. You the set of tools given to answer concisely."),
    ("human", "{input}")
])

# 2) Format it for a particular query
query = "What is the weather in New York"
formatted_prompt = prompt_template.format_prompt(input=query)

messages = formatted_prompt.to_messages()
model = llm.bind_tools(tools)

res = model.invoke(messages)
messages.append(res)

for tool_call in res.tool_calls:
    print("Tool call invoked")
    selected_tool = {"multiply":multiply,"power_of_two":power_of_two,"get_weather":get_weather}[tool_call['name'].lower()]
    tool_message = selected_tool.invoke(tool_call)
    messages.append(tool_message)

res = model.invoke(messages)
messages.append(res)

print(res.content)



Tool call invoked
{"output": "sunny", "temperature": 25}


In [1]:
from langchain.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

tools = [multiply]

In [ ]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model="llama3.1",
    openai_api_key="EMPTY",                      # Required param but unused
    openai_api_base="http://localhost:11434/v1", # Ollama endpoint
)

In [ ]:
from langgraph.prebuilt import create_react_agent


agent = create_react_agent(
    model=llm,
    tools=tools,
)

C:\Users\mihir\AppData\Local\Temp\ipykernel_10896\1865365590.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [12]:
from langchain_core.messages import HumanMessage
react_prompt = """
You are a Preventive Health Copilot.
Use the ReAct pattern: Think → Reason → Act.

User request: {query}

Steps:
1. Think through user goals.
2. Decide if you need health tips or general advice.
3. Answer clearly.

Respond in this format:
Thought:
Action:
Answer:
"""

# response = agent.invoke({
#     "messages": [
#         HumanMessage(content="What is 10 × 7?")
#     ]
# })

In [11]:
print(response["messages"][-1].content)

The result of multiplying 10 by 7 is 70.


In [13]:
resp = llm.invoke(react_prompt.format(query="How do I reduce hypertension risk?"))

In [14]:
print(resp.content)

Here's the response using the ReAct pattern:

**Think:**
User goal is to reduce hypertension risk, which implies they are concerned about high blood pressure and want to take preventive measures.
Current health tips available for reducing hypertension risk include lifestyle modifications (physical activity, diet, stress management), maintaining a healthy weight, not smoking, limiting alcohol consumption, managing salt intake.

**Reason:**
Deciding on the type of information to provide, I will offer specific actionable tips based on scientific evidence and consider the user's general understanding of health concepts. Since the goal is to reduce hypertension risk, providing general advice would be less effective than offering evidence-based tips tailored to this specific concern.

**Act:**
Based on steps 1 (think) and 2 (reason), here are my actionable recommendations:

1. **Physical Activity**: Aim for at least 150 minutes of moderate-intensity aerobic physical activity or 75 minutes of

In [ ]:
!pip list